<a href="https://colab.research.google.com/github/TaysTyas/Bakudan/blob/master/svd_train_test_split_without_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.metrics.pairwise import pairwise_distances
import math
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [96]:
# membaca dataset ratings dari file CSV
data = pd.read_csv('https://raw.githubusercontent.com/TaysTyas/Dataset-movie/main/Dataset%20MovieLens/ratings.csv')

ratings_base, ratings_test = train_test_split(data, test_size=0.3, random_state=42)

In [102]:
# Create user - item matrix (row: user, column: item) start at 0
train_matrix_pivot = ratings_base.pivot(index="userId", columns="movieId", values="rating")
test_matrix_pivot = ratings_test.pivot(index="userId", columns="movieId", values="rating")

train_matrix_pivot.shape, test_matrix_pivot.shape

((610, 8566), (610, 6124))

In [103]:
train_matrix_pivot.fillna(0,inplace=True)
test_matrix = test_matrix_pivot.fillna(0)


train_matrix = train_matrix_pivot.to_numpy()
test_matrix = test_matrix_pivot.to_numpy()

In [66]:
train_matrix

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 0. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [67]:
# Get u,s,vt by using SVD with k = 20
u, s, vt = svds(train_matrix, k = 6)

u.shape, s.shape, vt.shape

((610, 6), (6,), (6, 8566))

In [68]:
# Construction diagonal matrix
s_diag_matrix = np.diag(s)
s_diag_matrix

array([[116.30349098,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        , 121.04368064,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        , 129.47277452,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , 143.01213833,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        168.11693488,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        , 377.01014239]])

### We get the predictions by finding the dot product of the three matrices

In [80]:
# Get the predictions by finding the dot product of the three matrices
# np.dot(a,b) returns the dot product of a and b.
predictions_svd = np.dot(np.dot(u,s_diag_matrix),vt)
# predictions_svd
predicted_svd_df = pd.DataFrame(predictions_svd)
predicted_svd_df.head()

,0,1,2,3,4,5,6,7,8,9,...,8556,8557,8558,8559,8560,8561,8562,8563,8564,8565
0,2.258071,1.033783,0.596470,0.028228,0.105721,1.469329,0.427157,-0.006524,0.162914,1.752196,...,0.000291,-0.012057,-0.009377,-0.010717,-0.010717,-0.009377,-0.010717,-0.009377,-0.009377,-0.018759
1,0.192518,0.031749,-0.060882,-0.000666,-0.010155,0.111365,-0.035471,0.004116,-0.011861,-0.007344,...,0.000623,0.004119,0.003204,0.003662,0.003662,0.003204,0.003662,0.003204,0.003204,0.008273
2,0.026266,0.004061,0.014278,-0.001983,-0.016153,0.031751,-0.003294,-0.002700,0.005123,0.032237,...,-0.000058,-0.000252,-0.000196,-0.000224,-0.000224,-0.000196,-0.000224,-0.000196,-0.000196,-0.000873
3,0.780686,0.281424,0.210466,0.015191,0.075040,0.640446,0.221545,0.005609,0.036710,0.508707,...,0.000700,-0.006519,-0.005070,-0.005795,-0.005795,-0.005070,-0.005795,-0.005070,-0.005070,-0.007509
4,0.928866,0.596853,0.283934,0.059629,0.322726,0.428815,0.342580,0.110284,0.070430,0.865010,...,0.000579,-0.002480,-0.001929,-0.002204,-0.002204,-0.001929,-0.002204,-0.001929,-0.001929,-0.004848


In [81]:
predicted_svd_df = predicted_svd_df.round(3)
predicted_svd_df.head()

,0,1,2,3,4,5,6,7,8,9,...,8556,8557,8558,8559,8560,8561,8562,8563,8564,8565
0,2.258,1.034,0.596,0.028,0.106,1.469,0.427,-0.007,0.163,1.752,...,0.000,-0.012,-0.009,-0.011,-0.011,-0.009,-0.011,-0.009,-0.009,-0.019
1,0.193,0.032,-0.061,-0.001,-0.010,0.111,-0.035,0.004,-0.012,-0.007,...,0.001,0.004,0.003,0.004,0.004,0.003,0.004,0.003,0.003,0.008
2,0.026,0.004,0.014,-0.002,-0.016,0.032,-0.003,-0.003,0.005,0.032,...,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.001
3,0.781,0.281,0.210,0.015,0.075,0.640,0.222,0.006,0.037,0.509,...,0.001,-0.007,-0.005,-0.006,-0.006,-0.005,-0.006,-0.005,-0.005,-0.008
4,0.929,0.597,0.284,0.060,0.323,0.429,0.343,0.110,0.070,0.865,...,0.001,-0.002,-0.002,-0.002,-0.002,-0.002,-0.002,-0.002,-0.002,-0.005


In [82]:
# Get predicted ratings from predictions svd matrix
# trả về ma trận có rating khác 0, dựa trên ma trận test_matrix
# Ground truth target values.

predicted_ratings_svd = predictions_svd[test_matrix.nonzero()]

# predicted_ratings_svd

# Estimated target values.
test_truth = test_matrix[test_matrix.nonzero()]
# test_truth

In [116]:
# Hitung selisih absolut antara prediksi dan nilai aktual
absolute_errors = np.abs(predicted_ratings_svd - test_truth)

# Hitung Mean Absolute Error (MAE)
mae = np.mean(absolute_errors)
print(np.sum(absolute_errors))
print(98387.752/30251)
print("Mean Absolute Error (MAE):", mae)

98387.75231479725
3.252380152722224
Mean Absolute Error (MAE): 3.2523801631284006


In [83]:
test_truth, predicted_ratings_svd

(array([5. , 3. , 3. , ..., 5. , 2.5, 3. ]),
 array([ 0.19329018,  0.41590107,  0.11397174, ..., -0.02114875,
         3.13191889,  0.30257874]))

In [86]:
predicted_svd_df.columns[0]

0

In [75]:
test_matrix.nonzero()

(array([  0,   0,   0, ..., 609, 609, 609]),
 array([ 105,  161,  312, ..., 5944, 5961, 6033]))

In [76]:
len(predicted_ratings_svd), len(test_truth)

(30251, 30251)

In [63]:
# Calculate MAE
MAE = mean_absolute_error(predicted_ratings_svd,test_truth)

MAE

3.2523801631284006

In [104]:
test_matrix_pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,187595,189043,189111,189333,189547,189713,190213,190219,193567,193587
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Lampiran

In [110]:
lampiran_rating_prediksi = predicted_svd_df.reindex(test_matrix_pivot.index)
lampiran_rating_prediksi = predicted_svd_df.reindex(columns=test_matrix_pivot.columns)
lampiran_rating_prediksi.shape

(610, 6124)

In [112]:
# Membuat boolean mask untuk nilai aktual
bool_nilai_aktual = ~np.isnan(test_matrix_pivot)

In [113]:
# Memfilter nilai-nilai prediksi berdasarkan boolean mask
test_matrix_pivot[bool_nilai_aktual].to_excel('lampiran_aktual_tanpa_normalisasi.xlsx', index=True)
lampiran_rating_prediksi[bool_nilai_aktual].to_excel('lampiran_prediksi_tanpa_normalisasi.xlsx', index=True)

# ### Example recommendate for user 1

In [ ]:
user_id = 1
user_ratings = predictions_svd[user_id-1,:]

# get all movies not ratings of user 1
train_unkown_indices = np.where(train_matrix[user_id-1,:] == 0)[0]

# get recommendations for user 1
user_recommendations = user_ratings[train_unkown_indices]

# train_unkown_indices
# user_recommendations.shape
user_ratings, train_unkown_indices, user_recommendations


(array([ 2.25665486,  1.14365277,  1.05769068, ...,  0.        ,
         0.        , -0.02275194]),
 array([     1,      3,      4, ..., 193606, 193607, 193608]),
 array([ 1.14365277,  0.02493818,  0.17033358, ...,  0.        ,
         0.        , -0.02275194]))

In [ ]:
# get numbers of movie ratings of user 1
# user_id : 1 , row : 0
row_of_user = user_id -1;
train_matrix[train_matrix[[row_of_user]].nonzero()].shape

(173,)

In [ ]:
numbers_movie_rcm = 5 ;

print('\nRecommendations movies for user {} : \n'.format(user_id))

# Get recommendations of 5 movies with the highest scores

for movie_id in user_recommendations.argsort()[-numbers_movie_rcm:]:
    print(movie_id +1)


Recommendations movies for user 1 : 

1494
1136
1138
29
560
